## 优化方法

优化方法的目的就是找到能够使得 f(x) 的值达到最小值对应的权重

这个状态能够让模型的损失函数最小，而这个状态就是模型的权重

梯度下降法、牛顿法、拟牛顿法等，涉及的数学知识也更是不可胜数。同样的，PyTorch 也将优化方法进行了封装，我们在实际开发中直接使用即可，节省了大量的时间和劳动。

我们这节课要学习的梯度下降法

梯度向量的方向即为函数值增长最快的方向，梯度的反方向则是函数减小最快的方向。但是只有一个使曲线函数下降最快的梯度

为了得到最小的损失函数，我们要用梯度下降的方法使其达到最小值。

步子大小很重要

步子的大小，叫做学习率（learning rate）。因为步长的原因，理论上我们是不可能精确地走到目的地的，而是最后在最小值的某一个范围内不断地震荡，也会存在一定的误差，不过这个误差是我们可以接受的。


### 批量梯度下降法（Batch Gradient Descent，BGD）
线性回归模型是我们最常用的函数模型之一。假设对于一个线性回归模型，y 是真实的数据分布函数

$h_\theta(x) = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + ... + \theta_n x_n$

其中 $\theta$ 是模型的参数，$x$ 是输入特征。其中 $\theta$ 是 $h$ 的参数，也是我们需要通过梯度下降法来优化的参数。

$J(\theta) = \frac{1}{2m} \sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)})^2$

m 表示样本数量, $h_\theta(x^{(i)})$ 是模型对第 i 个样本的预测值，$y^{(i)}$ 是第 i 个样本的真实值。


首先我们对 $J(\theta)$ 关于 $\theta_j$ 求偏导数，得到：

$\frac{\partial J(\theta)}{\partial \theta_j} = \frac{1}{m} \sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)}) x_j^{(i)}$

我们就可以按照下降的方向去更新每个θ：

$\theta _j' = \theta_j - \alpha \frac{\partial J(\theta)}{\partial \theta_j} = \theta_j - \alpha \frac{1}{m} \sum_{i=1}^{m} (h_\theta(x^{(i)}) - y^{(i)}) x_j^{(i)}$

其中 $\alpha$ 是学习率（learning rate），控制着每次更新的步长。

$h_\theta(x^{(i)})$ 是模型对第 i 个样本的预测值，$y^{(i)}$ 是第 i 个样本的真实值。

在实际的开发中，往往有百万甚至千万数量级的样本，那这个更新的量就很恐怖了。所以就需要另一个办法，随机梯度下降法



In [6]:
# Give an example for the BGD
# y≈wx+b 拟合数据 
# (x,y)={(1,2),(2,4),(3,6)}。
import numpy as np

x = np.array([1., 2., 3.])
y = np.array([3., 5., 7.])
m = x.size

w, b = 0.0, 0.0
eta = 0.1
epochs = 1000

fx = lambda x: w * x + b

for _ in range(epochs):
    # y_hat is predicted value
    y_hat = fx(x)
    # y is the true value
    e = y - y_hat
    # 全量（batch=全部样本）梯度
    # dw 是关于 w 的梯度， db 是关于 b 的梯度
    # dw is dJ/dw, db is dJ/db 
    # J(w,b) = (1/2m) * Σ(y_i - (wx_i + b))^2
    # dJ/dw = -(1/m) * Σ x_i * (y_i - (wx_i + b))
    dw = -(1.0/m) * np.sum(x * e)
    db = -(1.0/m) * np.sum(e)
    # 更新 eta 是学习率
    w -= eta * dw
    b -= eta * db
    # 每次都是利用所有样本计算梯度并更新参数，如果样本量很大，计算量会很大！！！！

print(f"w={w:.4f}, b={b:.4f}")

w=2.0000, b=1.0000


### 随机梯度下降（Stochastic Gradient Descent，SGD）

$\theta _j' = \theta_j - \alpha (h_\theta(x^{(i)}) - y^{(i)}) x_j^{(i)}$

每训练一条数据就更新一条参数

SGD 虽然快，也会存在一些问题。就比如，训练数据中肯定会存在一些错误样本或者噪声数据，那么在一次用到该数据的迭代中，

优化的方向肯定不是朝着最理想的方向前进的，也就会导致训练效果（比如准确率）的下降。

最极端的情况下，就会导致模型无法得到全局最优，而是陷入到局部最优。

随机梯度下降方法选择了用损失很小的一部分精确度和增加一定数量的迭代次数为代价，换取了最终总体的优化效率的提高。

每次不用全部的数据，也不只用一条数据，而是用“一些”数据，这就是接下来我们要说的小批量梯度下降（Mini-batch Gradient Descent）。

![image11.png](mdfiles/image11.png)


In [8]:
# 还是写一个例子，方便理解 SGD
# 假设有4个样本，batch size = 4
X = np.array([[1.0], [2.0], [3.0], [4.0]])
Y = np.array([[3.0], [5.0], [7.0], [9.0]])

# 初始化参数
W = 0.0
b = 0.0
learning_rate = 0.01
num_epochs = 1000
m = X.shape[0]  # 样本数量
# 定义线性模型
def model(X):
    return W * X + b

for epoch in range(num_epochs):
    for i in range(m): # 遍历每个样本
        x_i = X[i]
        y_i = Y[i]
        
        # 前向
        y_pred = W * x_i + b
        error = y_i - y_pred
        
        # === SGD：基于单个样本更新 ===
        dW = -2 * x_i * error
        db = -2 * error
        
        # 参数更新
        W -= learning_rate * dW
        b -= learning_rate * db

print("SGD结果：")
print(W, b)
    

SGD结果：
[2.00000026] [0.99999917]


### 小批量梯度下降（Mini-batch Gradient Descent，MBGD）
Mini-batch 的方法是目前主流使用最多的一种方式，它每次使用一个固定数量的数据进行优化。batch size。batch size 较为常见的数量一般是 2 的 n 次方，比如 32、128、512 等，越小的 batch size 对应的更新速度就越快，反之则越慢，但是更新速度慢就不容易陷入局部最优。

比如图像任务 batch size 我们倾向于设置得稍微小一点，NLP 任务则可以适当的大一些。

[特别重要的参考链接， 必须看！！！！！！！](https://www.ruder.io/optimizing-gradient-descent/)

In [9]:
# 测试

import numpy as np

# 数据
X = np.array([[1.0], [2.0], [3.0], [4.0]])
Y = np.array([[3.0], [5.0], [7.0], [9.0]])

# 初始化参数
W = 0.0
b = 0.0
learning_rate = 0.01
num_epochs = 1000
batch_size = 2
m = X.shape[0]  # 样本数量 m = 4

# MBGD
for epoch in range(num_epochs):
    # 🚨 每轮训练前打乱数据（Mini-batch 常规操作）
    indices = np.random.permutation(m)
    X_shuffled = X[indices]
    Y_shuffled = Y[indices]

    # 按 batch size 进行分组
    for i in range(0, m, batch_size):
        X_batch = X_shuffled[i:i+batch_size]
        Y_batch = Y_shuffled[i:i+batch_size]

        # 前向
        Y_pred = W * X_batch + b
        error = Y_batch - Y_pred

        # === MBGD：使用 Mini-batch 样本计算梯度 ===
        dW = (-2 / batch_size) * np.sum(X_batch * error)
        db = (-2 / batch_size) * np.sum(error)

        # 参数更新
        W -= learning_rate * dW
        b -= learning_rate * db

print("Mini-batch GD 结果：")
print(f"W = {W:.4f}, b = {b:.4f}")


Mini-batch GD 结果：
W = 2.0002, b = 0.9993


## 挑战

1. **学习率的选择**：学习率过大可能导致训练过程发散，过小则可能导致收敛速度过慢。如何选择合适的学习率是一个重要的问题。
2. **学习率策略**：学习率策略试图通过退火等方法在训练过程中调整学习率，即根据预定义的策略或当目标函数在训练周期之间的变化低于阈值时降低学习率。然而，这些策略和阈值必须预先定义，因此无法适应数据集的特性。
3. **学习率调整**： 如果数据稀疏，并且特征的频率差异很大，我们可能不想将所有特征都更新到相同的程度，而是对很少出现的特征进行更大的更新。
4. **局部最优**：梯度下降法可能会陷入局部最优解，特别是在非凸函数中，这可能导致模型性能不佳。
5. **鞍点问题**：在高维空间中，梯度下降法可能会遇到鞍点，即梯度为零但不是极值点的情况，这会导致优化过程停滞。SDG (随机梯度下降) 通过引入噪声，有助于跳出鞍点。

Momentum、RMSProp、Adam 等优化算法通过引入动量、自适应学习率等机制，缓解了上述挑战，提高了优化效率和模型性能。Gradient descent optimization algorithms 这些算法被pytorch等深度学习框架广泛使用集成，现在的gap是如何根据具体任务选择合适的优化算法，以及如何调整其超参数以获得最佳性能。

问题：需要额外的数学优化算法吗，现在这些算法是不是已经足够好了？

目前，深度学习中的优化算法已经相当成熟，常用的优化算法如Adam、RMSProp和SGD等已经能够满足大多数应用场景的需求。这些算法通过引入动量、自适应学习率等机制，有效地提高了训练速度和模型性能。但是，随着深度学习模型的复杂性不断增加，仍然存在一些挑战，例如处理非凸优化问题、避免局部最优解以及提高收敛速度等。因此，研究和开发新的优化算法仍然是一个活跃的领域，特别是在特定应用场景下，可能需要针对性地设计优化方法以进一步提升性能。


```python

import LeNet #假定我们使用的模型叫做LeNet，首先导入模型的定义类
import torch.optim as optim #引入PyTorch自带的可选优化函数
...
net = LeNet() #声明一个LeNet的实例
criterion = nn.CrossEntropyLoss() #声明模型的损失函数，使用的是交叉熵损失函数
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
# 声明优化函数，我们使用的就是之前提到的SGD，优化的参数就是LeNet内部的参数，lr即为之前提到的学习率

#下面开始训练
for epoch in range(30): #设置要在全部数据上训练的次数
  
    for i, data in enumerate(traindata):
        #data就是我们获取的一个batch size大小的数据
  
        inputs, labels = data #分别得到输入的数据及其对应的类别结果
        # 首先要通过zero_grad()函数把梯度清零，不然PyTorch每次计算梯度会累加，不清零的话第二次算的梯度等于第一次加第二次的
        optimizer.zero_grad()
        # 获得模型的输出结果，也即是当前模型学到的效果
        outputs = net(inputs)
        # 获得输出结果和数据真正类别的损失函数
        loss = criterion(outputs, labels)
        # 算完loss之后进行反向梯度传播，这个过程之后梯度会记录在变量中
        loss.backward()
        # 用计算的梯度去做优化
        optimizer.step()
...


batch size 越大越好吗？

批量大小（batch size）的选择并不是越大越好，而是需要根据具体的任务和数据集来进行权衡。较大的 batch size 可以提高计算效率，因为它允许更好地利用硬件资源（如 GPU），并且在某些情况下可以加速训练过程。然而，过大的 batch size 可能会导致以下问题：

1. **内存限制**：较大的 batch size 需要更多的内存资源，可能会超出硬件的限制，导致训练过程无法进行。
2. **泛化能力下降**：较大的 batch size 可能会导致模型在训练集上表现良好，但在测试集上表现不佳，即泛化能力下降。这是因为较大的 batch size 可能会使模型更容易陷入局部最优解。
3. **收敛速度**：较小的 batch size 通常会导致更快的收敛速度，因为它引入了更多的噪声，有助于跳出局部最优解。然而，过小的 batch size 可能会导致训练过程不稳定。快的收敛速度，因为它引入了更多的噪声，有助于跳出局部最优解。然而，过小的 batch size 可能会导致训练过程不稳定。

模型训练的本质就是确定网络结构、设定损失函数与优化方法。

机器学习开发的3个步骤
1. 数据处理：主要包括数据清理、数据预处理、数据增强等。总之，就是构建让模型使用的训练集与验证集。
2. 模型训练：确定网络结构，确定损失函数与设置优化方法。
3. 模型评估：使用各种评估指标来评估模型的好坏。

模型训练3步：
1. 模型结构设计：例如，机器学习中回归算法、SVM 等，深度学习中的 VGG、ResNet、SENet 等各种网络结构，再或者你自己设计的自定义模型结构。
2. 给定损失函数：损失函数衡量的是当前模型预测结果与真实标签之间的差距。
3. 给定优化方法：与损失函数搭配，更新模型中的参数。